In [36]:
import os
import faiss
import pickle
import numpy as np

In [37]:
features_path = r'C:\Retrieval System\features\features_clip'

In [38]:
def normalize_data(data):
    # Kiểm tra nếu data là một vector 1 chiều
    if data.ndim == 1:
        normalized_data = data / np.linalg.norm(data)
    else:
        # Chuẩn hóa dữ liệu nếu là mảng 2 chiều
        normalized_data = data / np.linalg.norm(data, axis=1, keepdims=True)
    
    return normalized_data

In [39]:
save_name_file = []
features = []

for file in os.listdir(features_path):
    file_path = os.path.join(features_path, file)

    with open(file_path, "rb") as f:
        image_features_data = pickle.load(f) # 1 list, các phần tử là dict với image_name và features

        for item in image_features_data:
            features.append(item['features'])
            save_name_file.append(item['image_name'])

db = np.array(features)
print(db.shape)

d = db.shape[1]
print(d)
nb = db.shape[0]
# nq = query_beit.shape[0]

index = faiss.IndexFlatIP(d)   # build the index
print(index.is_trained)
index.add(normalize_data(db))  # add vectors to the index
print(index.ntotal)

save_path = os.path.join(r'C:\Retrieval System\Multimedia-Information-Retrieval\index', 'IndexFlatIP_clip.pkl')
# Lưu index và danh sách vào file
with open(save_path, "wb") as f:
    pickle.dump({"index": index, "map_index": save_name_file}, f)
# print('hehe')

(6392, 512)
512
True
6392


In [40]:
m = 32 # Số phần con
nbits = 8  # Số bit để mã hóa mỗi phần con

# Tạo chỉ số PQ
quantizer = faiss.IndexFlatIP(d)  # Index chuẩn để sử dụng làm bộ mã hóa (flat index)
index = faiss.IndexPQ(d, m, nbits)  # IndexPQ với d chiều, m phần con và nbits bits

# Đặt quantizer cho IndexPQ (bộ mã hóa)
index.train(normalize_data(db))  # Huấn luyện trên dữ liệu mẫu
index.add(normalize_data(db)) 

save_path = os.path.join(r'C:\Retrieval System\Multimedia-Information-Retrieval\index', 'IndexPQ_clip.pkl')
# Lưu index và danh sách vào file
with open(save_path, "wb") as f:
    pickle.dump({"index": index, "map_index": save_name_file}, f)

In [41]:
nlist = 12
# Tạo chỉ số PQ
quantizer = faiss.IndexFlatIP(d)  # Index chuẩn để sử dụng làm bộ mã hóa (flat index)
index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_INNER_PRODUCT)

# Đặt quantizer cho IndexPQ (bộ mã hóa)
index.train(normalize_data(db))  # Huấn luyện trên dữ liệu mẫu
index.add(normalize_data(db)) 

save_path = os.path.join(r'C:\Retrieval System\Multimedia-Information-Retrieval\index', 'IndexIVFFlat_clip.pkl')
# Lưu index và danh sách vào file
with open(save_path, "wb") as f:
    pickle.dump({"index": index, "map_index": save_name_file}, f)